In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
warnings.filterwarnings("ignore")


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
df.head()

In [ ]:
df.describe()

Here minimum value for Pregnancy is 0 which depicts "no pregnancy" but for other columns like Glucose, BloodPressure, SkinThickness, Insulin, BMI minimum value =0 doesn't make logical sense. 

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df['Glucose'].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
col=['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI']
for i in col:
    sns.boxplot(df[i])
    plt.show()

* **Glucose**- No outlier, so 0 can be replaced with mean value. 
* **BloodPressure**- Outliers at both end. At lower end min BP is 25 which is not logical and at higher end max value is 122 which is logical. So 0 can be replaced with median value. 
* **SkinThickness**-  Less outliers, so 0 can be replaced with mean value. 
* **Insulin**- Multiple outliers present, so 0 will be replaced by median. 
* **BMI**- There are considerable amount of outliers, so 0 will be replaced with median value.

In [ ]:
df.Glucose = df.Glucose.replace(0,df.Glucose.mean())
df.BloodPressure = df.BloodPressure.replace(0,df.BloodPressure.median())
df.SkinThickness = df.SkinThickness.replace(0,df.SkinThickness.mean())
df.Insulin = df.Insulin.replace(0,df.Insulin.median())
df.BMI = df.BMI.replace(0,df.BMI.median())
df.head(25)

In [ ]:
df.Outcome.value_counts()
sns.countplot(data = df , x='Outcome')


In [ ]:
labels= 'Diabetic','Not Diabetic'
plt.pie(df['Outcome'].value_counts(),labels=labels,autopct='%0.02f%%')
plt.legend()
plt.show()



In [ ]:
fig=plt.figure(figsize=(15,10))
sns.heatmap(df.corr(),annot=True)

Here BMI,Glucose and Age are correlated to Outcome.

In [ ]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split


XTrain,XTest,yTrain,yTest = train_test_split(X_scaler,y)

print(XTrain.shape)
print(XTest.shape)
print(yTrain.shape)
print(yTest.shape)

**SVC**

In [ ]:
from sklearn.svm import SVC

svc = SVC(probability=True)

grid = {'kernel':['rbf','linear'],'C':[0.01,0.1,1,0.001],'gamma':[0.1,0.01,0.2,0.4]}

grid_svc = GridSearchCV(svc,grid,scoring='accuracy',cv = 10,refit = True)

In [ ]:
grid_svc.fit(XTrain,yTrain)
print("Test Set Accuracy", grid_svc.score(XTest,yTest))
print("Best Score", grid_svc.best_score_)
print("Tuned Params",grid_svc.best_params_)

**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

knn = KNeighborsClassifier()

grid = {'n_neighbors':[5,10,15,25,30,50]}

grid_knn = GridSearchCV(knn,grid,scoring='accuracy',cv = 10,refit = True)

In [ ]:
grid_knn.fit(XTrain,yTrain)
print("Test Set Accuracy", grid_knn.score(XTest,yTest))
print("Best Score", grid_knn.best_score_)
print("Tuned Params",grid_knn.best_params_)

**Conclusion**

* During analyzation of data we encountered some outliers in the dataset, which were necessary to be replaced with meaningful values. 
* We have created K-Nearest Neighbors and SVC Models where SVC performed better than KNN.